In [ ]:
from cnnfeat import get_feats
import keras
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from keras.layers import *
import pandas as pd
import numpy as np
from position import Position
import matplotlib.pyplot as plt
%matplotlib inline
import imp
# imp.reload(cnnfeat)
from multiprocessing import Queue
from multiprocessing import Manager, Process, Pool
import os
from time import time
import pickle
import bcolz
from tqdm import tqdm 

In [ ]:
import bcolz
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from tqdm import tqdm
from time import time

In [ ]:
# file_name1 = '../slonik_data/positions.pkl'
# file_name2 = '../slonik_data/sf_scores.pkl'
# data = pd.concat([pd.read_pickle(file_name1), pd.read_pickle(file_name2)])
file_name = '/ssd/slonik_data/sf_scores_12dep.pkl'
data = pd.read_pickle(file_name)

In [ ]:
def fen_to_feats(i):
    return get_feats(Position.from_fen(data.iloc[i].fen))

In [ ]:
now = time()
n = len(data)
n_train = len(data) - 60000
n_valid = 60000
step=1000000
for i in range(0, n_train, step):
    for j in range(i, min(i+step, n_train)):
        features.append(fen_to_feats(j))
    features.flush()
for i in range(n_train, n, step):
    for j in range(i, min(i+step, n)):
        features_valid.append(fen_to_feats(j))
    features_valid.flush()    
print((time() - now) / 60, 'm')

In [ ]:
sf_scores12 = pd.read_pickle('/ssd/slonik_data/sf_scores_12dep.pkl')

# for num_workers in range(1,11):
# features_sf12 = bcolz.carray(np.empty((0,8,8,14)), rootdir='/ssd/slonik_data/features_sf12_origfen.bc', chunklen=1024, mode='w')
features_sf12_valid = bcolz.carray(np.empty((0,8,8,14)), rootdir='/ssd/slonik_data/features_sf12_origfen_valid.bc', chunklen=1024, mode='w')

n_valid = 60000
n = len(sf_scores12) - n_valid
step = 1024 * 8

now = time()

def do_work(i):
    return get_feats(Position.from_fen(sf_scores12.iloc[i].orig_fen))

for s in tqdm(range(n, n + n_valid, step)):
    num_workers = 7

    with Pool(num_workers) as p:
        res = p.map(do_work, range(s, min(s + step, n + n_valid)))

    features_sf12_valid.append(np.stack(res))
    features_sf12_valid.flush()

# print(num_workers, "workers", len(features_sf12), "results", (time() - now), 's')

In [ ]:
features_sf12 = bcolz.open(rootdir='/ssd/slonik_data/features_sf12_origfen.bc', mode='r')
print(len(features_sf12))
features_sf12_valid = bcolz.open(rootdir='/ssd/slonik_data/features_sf12_origfen_valid.bc', mode='r')
print(len(features_sf12_valid))
sf_scores12 = pd.read_pickle('/ssd/slonik_data/sf_scores_12dep.pkl')
len(sf_scores12)
# features_positions_sf3 = bcolz.open(rootdir='/ssd/slonik_data/features_positions+sf3.bc', mode='r')

In [ ]:
np.allclose(get_feats(Position.from_fen(sf_scores12.iloc[-60000 + 20].orig_fen)), features_sf12_valid[20])

In [ ]:
np.allclose(get_feats(Position.from_fen(sf_scores12.iloc[1000].orig_fen)), features_sf12[1000])